In [104]:
#importação bibliotecas
import pandas as pd
import pathlib
import win32com.client as win32

In [105]:
#importando base de dados
emails = pd.read_excel(r'Bases de Dados\Emails.xlsx')
lojas = pd.read_csv(r'Bases de Dados\Lojas.csv', encoding='latin1', sep=';')
vendas = pd.read_excel(r'Bases de Dados\Vendas.xlsx')
#display(emails)
#display(lojas)
#display(vendas)


,ID Loja,Loja
0,1,Iguatemi Esplanada
1,2,Shopping Midway Mall
2,3,Norte Shopping
3,4,Shopping Iguatemi Fortaleza
4,5,Shopping União de Osasco
5,6,Shopping Center Interlagos
6,7,Rio Mar Recife
7,8,Salvador Shopping
8,9,Rio Mar Shopping Fortaleza
9,10,Shopping Center Leste Aricanduva


In [106]:
#colocando a venda de acordo com a loja respectivamente
vendas = vendas.merge(lojas, on='ID Loja')
#display(vendas)

,Código Venda,Data,ID Loja,Produto,Quantidade,Valor Unitário,Valor Final,Loja
0,1,2019-01-01,1,Sapato Estampa,1,358,358,Iguatemi Esplanada
1,1,2019-01-01,1,Camiseta,2,180,360,Iguatemi Esplanada
2,1,2019-01-01,1,Sapato Xadrez,1,368,368,Iguatemi Esplanada
3,21,2019-01-02,1,Camisa Gola V Listrado,2,116,232,Iguatemi Esplanada
4,34,2019-01-02,1,Sapato Listrado,1,363,363,Iguatemi Esplanada
...,...,...,...,...,...,...,...,...
100994,69846,2019-12-25,8,Short Estampa,2,96,192,Salvador Shopping
100995,69846,2019-12-25,8,Tênis Estampa,5,256,1280,Salvador Shopping
100996,69850,2019-12-25,8,Calça Estampa,4,177,708,Salvador Shopping
100997,69972,2019-12-26,8,Terno Liso,3,720,2160,Salvador Shopping


In [107]:
#criando dicionario com a tabela de cada loja
dicionario_lojas = {}

for loja in lojas['Loja']:
    dicionario_lojas[loja] = vendas.loc[vendas['Loja']==loja, :]
    
print(dicionario_lojas)
    


{'Iguatemi Esplanada':       Código Venda       Data  ID Loja                 Produto  Quantidade  \
0                1 2019-01-01        1          Sapato Estampa           1   
1                1 2019-01-01        1                Camiseta           2   
2                1 2019-01-01        1           Sapato Xadrez           1   
3               21 2019-01-02        1  Camisa Gola V Listrado           2   
4               34 2019-01-02        1         Sapato Listrado           1   
...            ...        ...      ...                     ...         ...   
3975         69899 2019-12-25        1           Sapato Xadrez           4   
3976         69903 2019-12-25        1              Terno Liso           5   
3977         69928 2019-12-25        1                    Meia           1   
3978         69944 2019-12-25        1            Gorro Xadrez           5   
3979         69944 2019-12-25        1           Sapato Xadrez           2   

      Valor Unitário  Valor Final       

In [108]:
#colocando a data mais recente(da base de dados) para fazer o cálculo de todos os indicadores
dia_indicador = vendas['Data'].max()


In [109]:
#pegando o caminho da pasta e verificando os arquivos dentro dela
caminho_backup = pathlib.Path(r'Backup Arquivos Lojas')
arquivos_pasta_backup = caminho_backup.iterdir()

#pegando o nome de cada pasta dentro da pasta Backup Arquivos Lojas
lista_nomes_backup = [arquivo.name for arquivo in arquivos_pasta_backup]
    
    

In [110]:
#esse for irá criar uma pasta correspondente de cada loja e dentro dessas pastas irá criar uma planilha em excel de cada loja
for loja in dicionario_lojas:
    try:
        if loja not in lista_nomes_backup:
            nova_pasta = caminho_backup / loja
            nova_pasta.mkdir()
    except FileExistsError:
        pass
    #criando planilha excel para pasta de cada loja
    nome_arquivo = '{}_{}_{}_{}.xlsx'.format(dia_indicador.day, dia_indicador.month,dia_indicador.year, loja)
    local_arquivo = caminho_backup / loja / nome_arquivo
    dicionario_lojas[loja].to_excel(local_arquivo)
    

In [111]:
#definição das metas
meta_faturamento_dia = 1000
meta_faturamento_ano = 1650000
meta_qtprodutos_dia = 4
meta_qtprodutos_ano = 120
meta_ticketmedio_dia = 500
meta_ticketmedio_ano = 500

In [112]:
#criando as variáveis para localizar as vendas da loja do ano e do dia mais recente
for loja in dicionario_lojas: 
    vendas_loja = dicionario_lojas[loja]
    vendas_loja_dia = vendas_loja.loc[vendas_loja['Data']==dia_indicador, :]
    
    #calculando faturamento do ano e dia mais recente
    faturamento_ano = vendas_loja['Valor Final'].sum()
    
    faturamento_dia = vendas_loja_dia['Valor Final'].sum()
    
    qtde_produtos_ano = len(vendas_loja['Produto'].unique())
    
    qtde_produtos_dia = len(vendas_loja_dia['Produto'].unique())
    
    #ticket médio anual da loja
    valor_venda = vendas_loja.groupby('Código Venda').sum(numeric_only=True)
    ticket_medio_ano = valor_venda['Valor Final'].mean()
    
    #ticker médio dia recente
    valor_venda_dia = vendas_loja_dia.groupby('Código Venda').sum(numeric_only=True)
    ticket_medio_dia = valor_venda_dia['Valor Final'].mean()
    
    #Enviar por email para o gerente
    nome = emails.loc[emails['Loja']==loja, 'Gerente'].values[0]
    outlook = win32.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)
    mail.to = emails.loc[emails['Loja']==loja, 'E-mail'].values[0]
    mail.Subject = f'OnePage Dia{dia_indicador.day}/{dia_indicador.month} - Loja {loja}'
    
    #automatização dos botões verdes e vermelhos para demonstrar se as metas foram batidas
    if faturamento_dia >= meta_faturamento_dia:
        cor_fat_dia = 'green'
    else:
        cor_fat_dia = 'red'
    
    if faturamento_ano >= meta_faturamento_ano:
        cor_fat_ano = 'green'
    else:
        cor_fat_ano = 'red'
    
    if qtde_produtos_dia >= meta_qtprodutos_dia:
        cor_qtde_dia = 'green'
    else:
        cor_qtde_dia = 'red'
    
    if qtde_produtos_ano >= meta_qtprodutos_ano:
        cor_qtde_ano = 'green'
    else:
        cor_qtde_ano = 'red'
        
    if ticket_medio_dia >= meta_ticketmedio_dia:
        cor_ticket_dia = 'green'
    else:
        cor_ticket_dia = 'red'
        
    if ticket_medio_ano >= meta_ticketmedio_ano:
        cor_ticket_ano = 'green'
    else:
        cor_ticket_ano = 'red'
    
    mail.HTMLBody = f'''
    <p>Bom dia, {nome}</p>
    
    <p>O resultado de ontem <strong>({dia_indicador.day}/{dia_indicador.month})</strong> da <strong>Loja {loja}</strong>foi:</p>
    
    <table style="text-align: center">
      <tr>
        <th>Indicador</th>
        <th>Valor Dia</th>
        <th>Meta Dia</th>
        <th>Cenário Dia</th>
      </tr>
      <tr>
        <td>Faturamento</td>
        <td>R${faturamento_dia:.2f}</td>
        <td>R${meta_faturamento_dia:.2f}</td>
        <td><font color="{cor_fat_dia}">◙</font></td>
      </tr>
      <tr>
        <td>Diversidade de Produtos</td>
        <td>{qtde_produtos_dia:.2f}</td>
        <td>{meta_qtprodutos_dia:.2f}</td>
        <td><font color="{cor_qtde_dia}">◙</font></td>
      </tr>
      <tr>
        <td>Ticket Médio</td>
        <td>R${ticket_medio_dia:.2f}</td>
        <td>R${meta_ticketmedio_dia:.2f}</td>
        <td><font color="{cor_ticket_dia}">◙</font></td>
      </tr>
    </table>
    <br>
    <table style="text-align: center">
      <tr>
        <th>Indicador</th>
        <th>Valor Ano</th>
        <th>Meta Ano</th>
        <th>Cenário Ano</th>
      </tr>
      <tr>
        <td>Faturamento</td>
        <td>R${faturamento_ano:.2f}</td>
        <td>R${meta_faturamento_ano:.2f}</td>
        <td><font color="{cor_fat_ano}">◙</font></td>
      </tr>
      <tr>
        <td>Diversidade de Produtos</td>
        <td>{qtde_produtos_ano:.2f}</td>
        <td>{meta_qtprodutos_ano:.2f}</td>
        <td><font color="{cor_qtde_ano}">◙</font></td>
      </tr>
      <tr>
        <td>Ticket Médio</td>
        <td>R${ticket_medio_ano:.2f}</td>
        <td>R${meta_ticketmedio_ano:.2f}</td>
        <td><font color="{cor_ticket_ano}">◙</font></td>
      </tr>
    </table>
    
    <p>Segue em anexo a planilha com todos os dados para mais detalhes.</p>
    
    <p>Qualquer dúvida estou à disposição.</p>
    <p>Att., Gabriel Sobral</p>
    '''
    
    attachment = pathlib.Path.cwd() / caminho_backup / loja / f'{dia_indicador.day}_{dia_indicador.month}_{dia_indicador.year}_{loja}.xlsx'
    mail.Attachments.Add(str(attachment))
    
    mail.Send()
    

In [113]:
#criando rank das lojas em tempo diário e anual, criando planilha e colocando na pasta de backup
faturamento_lojas = vendas.groupby('Loja')['Valor Final'].sum().reset_index()
faturamento_lojas_ano = faturamento_lojas.sort_values(by='Valor Final', ascending=False)
#(faturamento_lojas_ano)

nome_arquivo = '{}_{}_Ranking Anual.xlsx'.format(dia_indicador.day, dia_indicador.month)
faturamento_lojas_ano.to_excel(caminho_backup / '{}'.format(nome_arquivo))

vendas_dia = vendas.loc[vendas['Data']==dia_indicador,:]
faturamento_lojas_dia = vendas_dia.groupby('Loja')[['Valor Final']].sum().reset_index()
faturamento_lojas_dia = faturamento_lojas_dia.sort_values(by='Valor Final', ascending=False)
#display(faturamento_lojas_dia)

nome_arquivo = '{}_{}_Ranking Dia.xlsx'.format(dia_indicador.day, dia_indicador.month)
faturamento_lojas_dia.to_excel(caminho_backup / '{}'.format(nome_arquivo))

,Loja,Valor Final
2,Iguatemi Campinas,1762419
24,Shopping Vila Velha,1731167
0,Bourbon Shopping SP,1726110
10,Rio Mar Recife,1722766
22,Shopping SP Market,1721763
6,Palladium Shopping Curitiba,1721120
4,Norte Shopping,1711968
9,Ribeirão Shopping,1707122
3,Iguatemi Esplanada,1699681
11,Rio Mar Shopping Fortaleza,1698430


,Loja,Valor Final
7,Salvador Shopping,3950
3,Novo Shopping Ribeirão Preto,3400
1,Center Shopping Uberlândia,2651
10,Shopping Eldorado,2391
8,Shopping Center Interlagos,1582
13,Shopping Recife,1366
2,Norte Shopping,1259
14,Shopping União de Osasco,1207
15,Shopping Vila Velha,937
0,Bourbon Shopping SP,676


In [114]:
#enviar o e-mail para  diretoria listando o rank das lojas
outlook = win32.Dispatch('outlook.application')

mail = outlook.CreateItem(0)
mail.To = emails.loc[emails['Loja']=='Diretoria', 'E-mail'].values[0]
mail.Subject = f'Ranking Dia {dia_indicador.day}/{dia_indicador.month}'
mail.Body = f"""
Prezados, bom dia

Melhor loja do Dia em Faturamento: Loja {faturamento_lojas_dia.iloc[0, 0]} com Faturamento R${faturamento_lojas_dia.iloc[0, 1]}
Pior loja do Dia em Faturamento: Loja {faturamento_lojas_dia.iloc[-1, 0]} com Faturamento R${faturamento_lojas_dia.iloc[-1, 1]}

Melhor loja do Ano em Faturamento: Loja {faturamento_lojas_ano.iloc[0, 0]} com Faturamento R${faturamento_lojas_ano.iloc[0, 1]}
Pior loja do Ano em Faturamento: Loja {faturamento_lojas_ano.iloc[-1, 0]} com Faturamento R${faturamento_lojas_ano.iloc[-1, 1]}

Segue em anexo os rankings do ano e do dia de todas as lojas.

Qualquer dúvida estou à disposição.

Att.,
Gabriel Sobral"""



# Anexos (pode colocar quantos quiser):
attachment  = pathlib.Path.cwd() / caminho_backup / f'{dia_indicador.day}_{dia_indicador.month}_Ranking Anual.xlsx'
mail.Attachments.Add(str(attachment))
attachment  = pathlib.Path.cwd() / caminho_backup / f'{dia_indicador.day}_{dia_indicador.month}_Ranking Dia.xlsx'
mail.Attachments.Add(str(attachment))


mail.Send()
print('E-mail da Diretoria enviado')

E-mail da Diretoria enviado
